In [8]:
from pystac import ItemCollection
from pystac_client import Client

%run _projections.ipynb


def get_collection(start_date, end_date, bbox, dst_path, max_cloud_cover=20, max_tile_count=4):
    """
    Gets a STAC collection
    Arguments:
        start_date: datetime
        end_date: datetime
        bbox: list as [min_lon, min_lat, max_lon, max_lat]
        step: directory name in /tmp where data is saved
        max_cloud_cover: integer percentage
    Returns:
        ItemCollection
    """

    assert end_date > start_date

    stac_date_format = '%Y-%m-%dT%H:%M:%SZ'
    stac_date_string = start_date.strftime(stac_date_format) + '/' + end_date.strftime(stac_date_format)

    # Open a catalog
    client = Client.open("https://earth-search.aws.element84.com/v0")

    # Get results for a collection in the catalog
    search = client.search(
        bbox=bbox,
        collections=['sentinel-s2-l2a-cogs'], 
        datetime=stac_date_string,
        sortby='-properties.datetime',
        query={
            "eo:cloud_cover":{
                "lt": str(max_cloud_cover)
            },
        },
    )

    print(f"\t{dst_path}: {search.matched()} items found")

    # limit number of images per Sentinel grid square
    items, items_count = [], {}
    for item in list(search.items()):
        square = item.properties['sentinel:grid_square']
        count = items_count.get(square, 0)
        if count < max_tile_count:
            items.append(item)
            items_count[square] = count + 1

    print(f'\t{items_count}')
            
    collection = ItemCollection(items=items)

    if len(items) == 0:
        raise ValueError("collection is empty")

    bbox_coverage = get_collection_bbox_coverage(collection, bbox)
    if bbox_coverage < 100:
        raise ValueError(f'coverage is {bbox_coverage}% of bbox')

    collection.save_object(dst_path)
    return collection

In [4]:
import os
from osgeo import gdal
from pystac import ItemCollection
from pystac_client import Client
import rasterio
import rasterio.crs
import rasterio.merge
import requests
from shapely.geometry import box, shape, Point
import xml.etree.ElementTree as ET

%run _constants.ipynb
%run _imagery.ipynb
%run _masking.ipynb
%run _projections.ipynb


def _get_scene_metadata(href):
    
    res = requests.get(href)
    data = res.content
    root = ET.fromstring(data)
            
    mean_angle = root.find(".//Mean_Sun_Angle")
    azimuth = float(mean_angle.find("AZIMUTH_ANGLE").text)
    zenith = float(mean_angle.find("ZENITH_ANGLE").text)
    
    return {
        'AZIMUTH_ANGLE': azimuth,
        'ZENITH_ANGLE': zenith
    }


def download_original_imagery(collection, bbox, bands, dst_dir, overwrite=False):

    bbox_poly_ll = box(*bbox)
    bbox_poly_ea = reproject_shape(bbox_poly_ll, "EPSG:4326", "EPSG:3857")

    blank_path = create_blank_tif(bbox_poly_ll, dst_dir)

    scenes_dict = {}
    for item in list(collection):
        print(f'\tdownloading... {item.id}')
        scenes_dict[item.id] = {}

        band_hrefs = [item.assets[band].href for band in bands]

        scenes_dict[item.id]['meta'] = _get_scene_metadata(item.assets['metadata'].href)
        
        # reproject bbox into projection of S2 item 
        item_epsg = f'EPSG:{item.properties["proj:epsg"]}'
        bbox_sw_utm = reproject_shape(Point(bbox[0], bbox[1]), init_proj="EPSG:4326", target_proj=item_epsg)
        bbox_ne_utm = reproject_shape(Point(bbox[2], bbox[3]), init_proj="EPSG:4326", target_proj=item_epsg)
        
        # calculate intersection between Sentinel scene and bounding box so that we can save data properly to tif
        scene_poly_ll = shape(item.geometry) # Polygon of the entire original image
        overlap_poly_ll = bbox_poly_ll.intersection(scene_poly_ll)
        overlap_poly_ea = reproject_shape(overlap_poly_ll, "EPSG:4326", "EPSG:3857")

        # get overlap in item UTM projection so we can save subset of data to tif
        overlap_bbox_ll = overlap_poly_ll.bounds
        overlap_sw_utm = reproject_shape(
            Point(overlap_bbox_ll[0], overlap_bbox_ll[1]),
            init_proj="EPSG:4326", 
            target_proj=item_epsg
        )
        overlap_ne_utm = reproject_shape(
            Point(overlap_bbox_ll[2], overlap_bbox_ll[3]), 
            init_proj="EPSG:4326", 
            target_proj=item_epsg
        )
        
        scene_dir = f'{dst_dir}/{item.id}'
        if not os.path.exists(scene_dir):
            os.mkdir(scene_dir)
            
        for s3_href in band_hrefs:
            
            band_name = s3_href.split('/')[-1].split('.')[0]
            band_path = f'{scene_dir}/{band_name}.tif'
            merged_path = f'{scene_dir}/{band_name}_merged.tif'
                        
            # TODO: remove this for deployment
            if os.path.exists(band_path) and not overwrite:
                scenes_dict[item.id][band_name] = merged_path
                continue

            with rasterio.open(s3_href) as s3_src:

                # create window and only read data from the window, rather than entire file
                window = rasterio.windows.from_bounds(
                    bbox_sw_utm.x, bbox_sw_utm.y, 
                    bbox_ne_utm.x, bbox_ne_utm.y, 
                    transform=s3_src.transform
                )

                s3_data = s3_src.read(1, window=window)
                height, width = s3_data.shape[0], s3_data.shape[1]

                new_transform = rasterio.transform.from_bounds(
                    overlap_bbox_ll[0], overlap_bbox_ll[1], 
                    overlap_bbox_ll[2], overlap_bbox_ll[3], 
                    width, height
                )
                
                kwargs = {
                    "driver": "GTiff",
                    "height": height,
                    "width": width,
                    "count": 1,
                    "dtype": s3_data.dtype,
                    "crs": rasterio.crs.CRS.from_epsg(4326),
                    "transform": new_transform
                }
                
                # alignment issues was due to projecting from UTM to 4326, so don't do that at all
                
                kwargs["nodata"] = NODATA_BYTE if band_name == "SCL" else NODATA_UINT16

                with rasterio.open(band_path, "w", **kwargs) as new_src:
                    new_src.write(s3_data, 1)

                res = 10 / (111.32 * 1000)
                gdal.Warp(band_path, band_path, xRes=res, yRes=res, outputBounds=overlap_poly_ll.bounds)
                
                merged_path = merge_tif_with_blank(band_path, blank_path, band_name, bbox_poly_ll, merged_path=merged_path)
                scenes_dict[item.id][band_name] = merged_path

    return scenes_dict
            

In [2]:
%run _constants.ipynb
%run _imagery.ipynb
%run _masking.ipynb

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# TODO: this can probably move to imagery

def get_processed_composites(collection, bbox, dst_dir, overwriteDownload=True, overwriteProcessing=True):
    """
    Downloads, processes and saves analysis ready Sentinel-2 images.
    Arguments:
        collection: ...
        TODO:COMPLETE ME
    Returns:
        Dictionary of band to composite path mappings.
    """

    original_scenes_dict = download_original_imagery(collection, bbox, S2_BANDS, dst_dir, overwrite=overwriteDownload)
        
    masked_scenes_dict = {}
    for scene in original_scenes_dict:
        print(f'\tmasking... {scene}')
        scene_dir = f'{dst_dir}/{scene}'
        scene_dict = original_scenes_dict[scene]
        masked_scenes_dict[scene] = save_cloud_masked_images(scene_dict, scene_dir, overwrite=overwriteProcessing)  
        
    composite_dict = {}
    for band in S2_BANDS:
        if band == "SCL": continue

        print(f'\tstacking... {band}')
        
        masked_paths = [masked_scenes_dict[scene][band] for scene in masked_scenes_dict]
        stack_path = create_band_stack(band, masked_paths, dst_dir)
        
        print(f'\tcompositing... {band}')

        composite_path = create_composite_dask(band, stack_path, dst_dir, method="median", overwrite=overwriteProcessing)        
        composite_dict[band] = composite_path

    return composite_dict
